In [1]:
# 创建关系矩阵
def create_relation_matrix(List, n):
    adjacent_matrix = create_matrix(n, 0)
    for relation in List:
        adjacent_matrix[relation[0]][relation[1]] = 1
    return adjacent_matrix

# 输出列表
def printf(List):
    for x in List:
        print (x)
        
# 列表去重
def list_unique(List):
    new_list = []
    for id in List:
        if id not in new_list:
            new_list.append(id)
    return new_list

def create_matrix(number, amount):  #创建number×number的方阵，每个元素是amount
    matrix = []
    for i in range(0, number):
        tmp = []
        for j in range(0, number):
            tmp.append(amount)
        matrix.append(tmp)
    return matrix

# 查找包含该元素的所有位置
def find_index(List, node):
    return [i for i, j in enumerate(List) if j == node]

# 获取模块度
def get_modularity(node_list, node_club, club_list, node_matrix):
    uni = list_unique(club_list) #社区列表去重
    # 更新社团位置
    for node in uni:
        idices = find_index(club_list, node) #在社区中找到某顶点的位置
        for i in idices:
            node_club[i] = uni.index(node)
    Q=0
    #m = sum([sum(node) for node in node_matrix])/2  # 网络的边的数目
    m = sum([sum(node) for node in node_matrix])  # 网络的边的数目
    k = len(list_unique(node_club))  # 当前社团数目
    e = create_matrix(k, 0)  # 构造0矩阵  #k维0矩阵 （方阵）
    for i in range(k):
        idx = find_index(node_club, i)
        labelsi = idx
        for j in range(k):
            idx = find_index(node_club, j)
            labelsj = idx
            for ii in labelsi:
                for jj in labelsj:
                    e[i][j] = e[i][j]+node_matrix[ii][jj]  # e[i][j]代表i社团与j社团之间有多少连接
    e = [[float(j)/ m for j in i] for i in e]
    a_out = []  #行和  出度
    a_in = []  #列和，入度
    e1=pd.DataFrame(e)
    for i in range(k):
        ai_out = sum(e[i])
        ai_in = sum(e1.iloc[:,i])
        a_out.append(ai_out)
        a_in.append(ai_in)
        Q = Q + e[i][i]-ai_out*ai_in
    return Q, e, a_out,a_in, node_club
def fast_newman(node_list, List,n,divide_num):
    adjacent_matrix=create_relation_matrix(List, n)
    n = len(adjacent_matrix)   #节点的数目
    max_id = n
    Z = []
    # 初始划分，node_list是节点标号，node_club是社团标号的变换，club_list是社团标号
    node_club = [0 for i in range(n)]
    club_list = [i for i in range(n)]
    step = 1
    Q_total=[]
    club_total=[]
    while len(list_unique(club_list)) != 1:  # 计算满足条件的个数
        Q, e, a_out,a_in, node_club = get_modularity(node_list, node_club, club_list, adjacent_matrix)
        k = len(e)  # 社团数目
        DeltaQs = []
        DeltaQs_i = []
        DeltaQs_j = []
        for i in range(k):
            for j in range(k):
                if i != j:
                    #DeltaQ = 2*(e[i][j]-a[i]*a[j])
                    DeltaQ = e[i][j]+e[j][i]-a_out[i]*a_in[j]-a_out[j]*a_in[i]
                    DeltaQs.append(DeltaQ)
                    DeltaQs_i.append(i)
                    DeltaQs_j.append(j)
        maxDeltaQ = max(DeltaQs)  # 选择最大Q值的社团进行合并
        id_club = DeltaQs.index(maxDeltaQ)
        i = DeltaQs_i[id_club]
        j = DeltaQs_j[id_club]
        max_id = max_id + 1
        c_id1 = find_index(node_club, i)  # 获取社团i的标号
        c_id2 = find_index(node_club, j)  # 获取社团j的标号
        id1 = list_unique([club_list[item] for item in c_id1])  # 找到社团i的所有节点
        id2 = list_unique([club_list[item] for item in c_id2])  # 找到社团j的所有节点
        for item in c_id1:
            club_list[item] = max_id

        for item in c_id2:
            club_list[item] = max_id
        Z.append([id1, id2, len(c_id1+c_id2)])
        step = step + 1
        result_name = []
        result_index = []
        for item in list_unique(club_list):
            tmp = find_index(club_list, item)
            result_name.append([node_list[t] for t in tmp])
            result_index.append(tmp)
        Q1=get_modularity(node_list, node_club, club_list, adjacent_matrix)[0]
        Q_total.append(Q1)
        club_total.append(result_name)
        if len(result_name) == 1:
            break
            #完全终止当前循环
    club_link=[]
    for item in List:
        if club_list[item[0]]!=club_list[item[1]]:
            club_link.append(item)
    club_total=club_total[Q_total.index(max(Q_total))]
    Q_total=max(Q_total)
    

    return Q_total,club_total
if __name__ == '__main__':
    import pandas as pd
    import numpy as np
    import pandas as pd

    #data=pd.read_table(r"D:\看论文\6.数据\5.celegans_n306\celegans_n306.txt",sep=' ',header=None)
    #data=pd.read_csv(r"D:\看论文\6.数据\4.Friendship-network_data_2013.csv\Friendship-network_data_2013.csv",header=None)
    #data=pd.read_table(r"D:\看论文\6.数据\6.download.tsv.convote\convote\out.convote.txt",sep=' ',header=None)
    #data=pd.read_table(r"D:\看论文\6.数据\7.download.tsv.moreno_innovation\moreno_innovation\out.moreno_innovation_innovation.txt",sep=' ',header=None)
    #data=pd.read_csv(r"D:\看论文\6.数据\download.tsv.maayan-foodweb\maayan-foodweb\out.maayan-foodweb.txt",sep=' ',header=None)
    #data=pd.read_csv(r"D:\看论文\6.数据\download.tsv.wiki_talk_ht\wiki_talk_ht\wiki_talk.txt",sep='\t|,| ',header=None)

    
    
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-1\l_data500边排序.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-2\l_data500边排序.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-3\l_data500边排序.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-4\l_data500边排序.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-5\l_data500边排序.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-6\l_data500边排序.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-7\l_data500边排序.csv",header=None)
    data=pd.read_csv(r"D:\1.看论文0\14.小型LFR\LFR-8\l_data500边排序.csv",header=None)
    
    a=[]
    for i in data[0]:
        if i not in a:
            a.append(i)
    for i in data[1]:
        if i not in a:
            a.append(i)
    a=sorted(a)
    List=[]
    for i ,j in zip(data[0],data[1]):
        i=a.index(i)
        j=a.index(j)
        List.append((i,j))
    node_nums=len(a) # 该数据集的顶点数目
    node_list = a
    import time
    start=time.time()
    res=fast_newman(node_list, List, node_nums, node_nums)
    print(res)
    end=time.time()
    t=end-start
    print("用时：",t)

(0.6879600000000001, [[0, 19, 28, 92, 98, 101, 131, 140, 158, 160, 170, 183, 195, 208, 215], [1, 52, 73, 77, 79, 89, 129, 130, 132, 147, 176, 187, 199, 201, 224, 238, 249], [2, 4, 21, 31, 33, 54, 67, 85, 86, 87, 90, 104, 109, 113, 122, 143, 151, 153, 180, 189, 200, 202, 206, 236, 259, 267], [3, 13, 14, 16, 24, 29, 36, 40, 48, 55, 63, 65, 68, 74, 75, 83, 94, 106, 111, 123, 134, 146, 167, 177, 186, 192, 197, 213], [5, 10, 22, 41, 42, 46, 47, 53, 60, 71, 76, 108, 114, 119, 135, 144, 162, 274, 275], [6, 26, 38, 66, 69, 110, 118, 141, 156, 178, 179, 184, 193, 211, 212, 239], [7, 27, 78, 95, 102, 112, 148, 166, 173, 188, 214, 217, 233, 234, 235, 256, 263, 264, 266, 277, 279, 280], [8, 56, 58, 62, 107, 133, 139, 149, 191, 205, 210, 219, 220, 258, 271], [9, 23, 30, 49, 82, 103, 105, 138, 152, 168, 172, 203, 223, 226, 230, 250, 255, 257, 262, 265, 281], [11, 35, 39, 59, 70, 96, 117, 127, 137, 164, 231, 248, 252], [12, 20, 50, 80, 84, 100, 126, 155, 157, 216, 225, 229, 232, 254, 261, 270], [15, 